<a href="https://colab.research.google.com/github/ShivaShirsath/ImageProcessing/blob/master/mp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import math

class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0

    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [ ]:
import cv2
import imageio

# Create tracker object
tracker = EuclideanDistTracker()

cap = cv2.VideoCapture("highway.mp4")

# Object detection from Stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

# Create GIF writer
gif_writer_roi = imageio.get_writer('roi.gif', mode='I', fps=10)
gif_writer_frame = imageio.get_writer('frame.gif', mode='I', fps=10)
gif_writer_mask = imageio.get_writer('mask.gif', mode='I', fps=10)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape

    # Extract Region of interest
    roi = frame[340: 720,500: 800]

    # 1. Object Detection
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 100:
            #cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h = cv2.boundingRect(cnt)
            detections.append([x, y, w, h])

    # 2. Object Tracking
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.putText(roi, str(id), (x, y - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

    # Write frame to GIF
    gif_writer_roi.append_data(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
    gif_writer_frame.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    gif_writer_mask.append_data(cv2.cvtColor(mask, cv2.COLOR_BGR2RGB))

    key = cv2.waitKey(0)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()
# Close GIF writer
gif_writer_roi.close()
gif_writer_frame.close()
gif_writer_mask.close()

{0: (163, 79)}
{1: (161, 122)}
{1: (163, 127)}
{1: (162, 136)}
{1: (163, 149)}
{1: (160, 144)}
{1: (159, 153)}
{1: (162, 174)}
{1: (160, 168), 2: (158, 148)}
{1: (159, 180)}
{1: (158, 190)}
{1: (158, 198)}
{1: (158, 210)}
{1: (157, 220)}
{1: (156, 232)}
{1: (156, 244)}
{1: (156, 256)}
{1: (155, 269)}
{1: (154, 284)}
{1: (153, 298)}
{1: (152, 314)}
{1: (151, 330)}
{1: (150, 342)}
{1: (148, 351)}
{1: (145, 360)}
{1: (143, 369)}
{3: (207, 20)}
{3: (207, 24)}
{3: (208, 26)}
{3: (208, 30)}
{3: (208, 33)}
{3: (209, 28)}
{3: (208, 40)}
{3: (208, 35)}
{3: (209, 38)}
{3: (209, 41)}
{3: (208, 53)}
{3: (209, 48)}
{3: (210, 52)}
{3: (210, 64)}
{3: (212, 45)}
{3: (210, 58)}
{3: (210, 63)}
{3: (210, 66)}
{3: (211, 70)}
{3: (211, 74)}
{3: (211, 79)}
{3: (210, 83)}
{3: (211, 87)}
{3: (212, 90)}
{3: (212, 96)}
{3: (212, 100)}
{3: (212, 105), 4: (159, 32)}
{3: (212, 105), 4: (159, 30)}
{3: (213, 110), 4: (159, 30)}
{3: (213, 110), 4: (159, 39)}
{3: (213, 116), 4: (159, 39)}
{3: (213, 116), 4: (158, 46)}

<table>
<thead>
<tr>
<th style="text-align:center">Region of interest</th>
<th style="text-align:center">Frame</th>
<th style="text-align:center">Mask</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:center"><img src="roi.gif" alt=""></td>
<td style="text-align:center"><img src="frame.gif" alt=""></td>
<td style="text-align:center"><img src="mask.gif" alt=""></td>
</tr>
</tbody>
</table>
